In [51]:
import os
import pandas as pd
import time
from datetime import datetime
from pvoutput import *

In [23]:
INPUT_PV_LIST_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_listing_metadata.hdf'
OUTPUT_METADATA_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_metadata.csv'

In [2]:
pv_systems = pd.read_hdf(INPUT_PV_LIST_FILENAME, 'metadata')

In [42]:
pv_systems.head()

,Array Tilt,Inverter,Location,Orientation,Panels,Shade,system_capacity,system_name,Outputs,Generation,Average,Efficiency,system_capacity_kw,efficiency_kWh_per_kW,system_1_tilt_degrees,system_2_tilt_degrees
system_id,,,,,,,,,,,,,,,,
26965,11.0°,Enphase M215,United Kingdom CV47,South 0.0°,336x250W Q Cells Pro-G3 250,Low,84.000kW,mfl_phc,1813 days,399.238MWh,220.208kWh,2.654kWh/kW,84.00,2.654,11.0,NaN
24768,NaN,aurora trio,United Kingdom OX7,South 180.0°,1050x240W qidong,No,252.000kW,mfl_qfa,446 days,307.029MWh,688.405kWh,2.732kWh/kW,252.00,2.732,NaN,NaN
11542,15.0°,SMA TRI-Power 1700 TL,United Kingdom PE11,South 180.0°,200x250W Emmvee ES 230 M60 B,No,50.000kW,Wray Farms Solar System,2437 days,293.684MWh,120.510kWh,2.477kWh/kW,50.00,2.477,15.0,NaN
66991,40.0°,Unknown,United Kingdom HR8,South 180.0° / South West 225.0°,152x325W + 80x325W Unknown,No,75.400kW,Ledbury Community Hospital,1434 days,279.902MWh,195.190kWh,2.603kWh/kW,75.40,2.603,40.0,NaN
5116,15.0°,Fronius CL36; Datamanager 2.0,United Kingdom OX7,South 180.0°,182x235W Q.Base-G2 235,No,42.770kW,mfl_scf,2538 days,267.470MWh,105.386kWh,2.493kWh/kW,42.77,2.493,15.0,NaN


In [68]:
# Get list of systems we got from the PVOutput.org API search
pv_sys_api_search = pd.read_csv(
    '/home/jack/data/pvoutput.org/raw/uk_pv_systems.csv',
    index_col='system_id',
    usecols=['system_id'])

In [69]:
if os.path.exists(OUTPUT_METADATA_FILENAME):
    output_metadata = pd.read_csv(OUTPUT_METADATA_FILENAME, index_col='system_id', usecols=['system_id'])
    systems_already_processed = output_metadata.index
    header = False
else:
    systems_already_processed = []
    header = True
    
print(len(systems_already_processed), 'systems already processed.')

2559 systems already processed.


In [73]:
pv_systems_to_process = list(
    (set(pv_systems.index).union(pv_sys_api_search.index)) - 
    set(systems_already_processed))
print(len(pv_systems_to_process), 'PV systems left to process.')

1471 PV systems left to process.


In [67]:
n = len(pv_systems_to_process)
for i, pv_system_id in enumerate(pv_systems_to_process):
    print('\r', '{:>4d}'.format(i), 'of', n, '| ID =', pv_system_id, end='', flush=True)
    pv_metadata = get_pv_metadata(pv_system_id).to_frame().T.set_index('system_id')
    pv_metadata.to_csv(
        OUTPUT_METADATA_FILENAME,
        mode='a',
        header=header)
    header = False

    0 of 1971 | ID = 26572
Bad status code returned: 403, Forbidden 403: Exceeded 300 requests per hour
Waiting 54 minutes...
Done waiting!  Retrying...
  300 of 1971 | ID = 3074
Bad status code returned: 403, Forbidden 403: Exceeded 300 requests per hour
Waiting 56 minutes...
Done waiting!  Retrying...
  600 of 1971 | ID = 4185
Bad status code returned: 403, Forbidden 403: Exceeded 300 requests per hour
Waiting 57 minutes...
Done waiting!  Retrying...
  900 of 1971 | ID = 37689
Bad status code returned: 403, Forbidden 403: Exceeded 300 requests per hour
Waiting 56 minutes...
Done waiting!  Retrying...
 1200 of 1971 | ID = 30248
Bad status code returned: 403, Forbidden 403: Exceeded 300 requests per hour
Waiting 57 minutes...
Done waiting!  Retrying...
 1500 of 1971 | ID = 6555
Bad status code returned: 403, Forbidden 403: Exceeded 300 requests per hour
Waiting 56 minutes...
Done waiting!  Retrying...
 1800 of 1971 | ID = 40277
Bad status code returned: 403, Forbidden 403: Exceeded 300

In [95]:
!head $OUTPUT_METADATA_FILENAME

system_id,system_name,system_size_watts,postcode,number_of_panels,panel_power_watts,panel_brand,num_inverters,inverter_power_watts,inverter_brand,orientation,array_tilt_degrees,shade,install_date,latitude,longitude,status_interval_minutes,number_of_panels_secondary,panel_power_watts_secondary,orientation_secondary,array_tilt_degrees_secondary
19397,AndyT's,3900,BS22,20,195,ZN Shine 195w,1,3500,Kaco 4202,S,30.0,No,2011-11-21 00:00:00,51.36,-2.92,5,0,0,,
8195,Kilmarnock Roof,3750,KA3,15,250,Sanyo 250 HIT,1,4000,Omniksol,S,30.0,No,2011-11-07 00:00:00,55.64,-4.49,10,0,0,,
8200,Flat 5,3430,E8,14,245,,1,3000,sb3000,S,25.0,Low,2011-12-12 00:00:00,51.54,-0.06,5,0,0,,
8204,Sooper-Dooper Solar,2940,GU2,12,245,SunTech STP245S-20/Wd,1,3000,Kaco Powador 3002,S,19.0,No,2012-05-11 00:00:00,51.24,-0.59,10,0,0,,
8205,58GPR,4000,BS48,16,250,Sanyo component Europe GmbH,1,50,Sma sunny boysb3800v,S,,No,2011-11-10 00:00:00,51.42,-2.74,10,0,0,,
32783,olaf-UK,3780,B92,14,270,canadian solar CS6P-270MM,1,3600,S